In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Project6_Pdm_Predict the number of remaining operational cycles before failure for Turbofan engine/unzipped_data/Turbofan engine/RUL_FD001.txt", sep='\s+', header=None)


In [ ]:
file_path = "/content/drive/MyDrive/Project6_Pdm_Predict the number of remaining operational cycles before failure for Turbofan engine/unzipped_data/Turbofan engine/train_FD001.txt"

columns = ['unit_number', 'cycle'] + \
          [f'op_setting_{i}' for i in range(1, 4)] + \
          [f'sensor_measurement_{i}' for i in range(1, 22)]

df = pd.read_csv(file_path, sep='\s+', header=None, names=columns)

df.head()


,unit_number,cycle,op_setting_1,op_setting_2,op_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [ ]:

# Step 1: Get max cycle for each engine
rul = df.groupby("unit_number")["cycle"].max().reset_index()
rul.columns = ["unit_number", "max_cycle"]

# Step 2: Merge back to main df
df = df.merge(rul, on="unit_number", how="left")

# Step 3: Calculate RUL
df["RUL"] = df["max_cycle"] - df["cycle"]

# Step 4: Drop helper column
df.drop("max_cycle", axis=1, inplace=True)

# Step 5: Check the updated data
df.head()


,unit_number,cycle,op_setting_1,op_setting_2,op_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [ ]:
# Check standard deviation of each column
low_variance = df.loc[:, 'sensor_measurement_1':'sensor_measurement_21'].std()
low_variance_sensors = low_variance[low_variance < 1e-2].index.tolist()

print("⚠️ Low-variance sensors:", low_variance_sensors)


⚠️ Low-variance sensors: ['sensor_measurement_1', 'sensor_measurement_5', 'sensor_measurement_6', 'sensor_measurement_10', 'sensor_measurement_16', 'sensor_measurement_18', 'sensor_measurement_19']


In [ ]:
df.drop(columns=[
    'sensor_measurement_1',
    'sensor_measurement_5',
    'sensor_measurement_6',
    'sensor_measurement_10',
    'sensor_measurement_16',
    'sensor_measurement_18',
    'sensor_measurement_19'
], inplace=True)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Get only feature columns to scale
features_to_scale = df.columns.difference(['unit_number', 'cycle', 'RUL'])

scaler = MinMaxScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])

df.head()


,unit_number,cycle,op_setting_1,op_setting_2,op_setting_3,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_7,sensor_measurement_8,sensor_measurement_9,sensor_measurement_11,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_17,sensor_measurement_20,sensor_measurement_21,RUL
0,1,1,0.459770,0.166667,0.0,0.183735,0.406802,0.309757,0.726248,0.242424,0.109755,0.369048,0.633262,0.205882,0.199608,0.363986,0.333333,0.713178,0.724662,191
1,1,2,0.609195,0.250000,0.0,0.283133,0.453019,0.352633,0.628019,0.212121,0.100242,0.380952,0.765458,0.279412,0.162813,0.411312,0.333333,0.666667,0.731014,190
2,1,3,0.252874,0.750000,0.0,0.343373,0.369523,0.370527,0.710145,0.272727,0.140043,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375,189
3,1,4,0.540230,0.500000,0.0,0.343373,0.256159,0.331195,0.740741,0.318182,0.124518,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386,188
4,1,5,0.390805,0.333333,0.0,0.349398,0.257467,0.404625,0.668277,0.242424,0.149960,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502,187


In [ ]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# 🛠️ Slope function
def compute_slope(values):
    x = np.arange(len(values))
    A = np.vstack([x, np.ones(len(x))]).T
    m, _ = np.linalg.lstsq(A, values, rcond=None)[0]
    return m

# ✅ Define column names
column_names = ['unit_number', 'time_in_cycles', 'operational_setting_1', 'operational_setting_2',
                'operational_setting_3'] + [f'sensor_measurement_{i}' for i in range(1, 22)]

# ✅ Load training data
df = pd.read_csv("/content/drive/MyDrive/Project6_Pdm_Predict the number of remaining operational cycles before failure for Turbofan engine/unzipped_data/Turbofan engine/train_FD001.txt",
                 sep='\s+', header=None)
df.columns = column_names

# ✅ Calculate RUL
rul_df = df.groupby('unit_number')['time_in_cycles'].max().reset_index()
rul_df.columns = ['unit_number', 'max_cycles']
df = df.merge(rul_df, on='unit_number')
df['RUL'] = df['max_cycles'] - df['time_in_cycles']
df.drop(columns=['max_cycles'], inplace=True)

# ✅ Feature Engineering
features = [f'sensor_measurement_{i}' for i in range(1, 22)]
agg_data = []

for engine_id in df['unit_number'].unique():
    engine_data = df[df['unit_number'] == engine_id].copy()
    for i in [-30, -25, -20, -15, -10]:
        if len(engine_data) < abs(i):
            continue
        window = engine_data.iloc[i:]
        row = {'unit_number': engine_id, 'RUL': window['RUL'].values[0]}
        for col in features:
            values = window[col].values
            row[f'{col}_mean'] = values.mean()
            row[f'{col}_slope'] = compute_slope(values)
            row[f'{col}_last'] = values[-1]
        agg_data.append(row)

# ✅ Final Dataset
agg_df = pd.DataFrame(agg_data)
X = agg_df.drop(['unit_number', 'RUL'], axis=1)
y = agg_df['RUL']

# ✅ Data Check
if len(np.unique(y)) == 1:
    raise ValueError("😵 All RULs are the same — can't train.")
else:
    print(f"✅ Unique RULs found: {np.unique(y).shape[0]} levels")

# ✅ Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Models
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('lgbm', LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)),
    ('cat', CatBoostRegressor(verbose=0, iterations=500, learning_rate=0.05, depth=6, random_state=42))
]
meta_model = Ridge(alpha=1.0)

stacked_model = StackingRegressor(estimators=base_models, final_estimator=meta_model, n_jobs=-1)

# ✅ Train
stacked_model.fit(X_train, y_train)

# ✅ Predict + Evaluate
y_pred = stacked_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"📊 MAE: {mae:.2f}")
print(f"📈 RMSE: {rmse:.2f}")
print(f"🎯 R² Score: {r2:.2f}")


✅ Unique RULs found: 5 levels
📊 MAE: 0.01
📈 RMSE: 0.02
🎯 R² Score: 1.00


In [ ]:
!pip install joblib

In [ ]:
import joblib
joblib.dump(stacked_model, "rul_model.pkl")

from google.colab import files
files.download("rul_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import numpy as np
import pandas as pd
import joblib
stacked_model = joblib.load("rul_model.pkl")

# 🔁 Slope function
def compute_slope(values):
    x = np.arange(len(values))
    A = np.vstack([x, np.ones(len(x))]).T
    m, _ = np.linalg.lstsq(A, values, rcond=None)[0]
    return m

# 🔮 Prediction function
def predict_rul_from_arrays(*sensor_arrays):
    features = {}
    for i, values in enumerate(sensor_arrays):
        values = np.array(values).flatten()
        features[f'sensor_measurement_{i+1}_mean'] = values.mean()
        features[f'sensor_measurement_{i+1}_slope'] = compute_slope(values)
        features[f'sensor_measurement_{i+1}_last'] = values[-1]
    df = pd.DataFrame([features])
    prediction = stacked_model.predict(df)[0]
    return round(prediction, 2)

# 📂 File-based prediction
def predict_rul_from_file(file):
    try:
        df = pd.read_csv(file.name, header=None)
        sensors = df.values.T  # transpose to [21, 30]
        return predict_rul_from_arrays(*sensors)
    except Exception as e:
        return f"❌ Error: {str(e)}"

# 🎲 Random dummy data generator
def generate_dummy_data():
    return [np.round(np.random.normal(loc=50, scale=10, size=(1, 30)), 2) for _ in range(21)]

# 🧠 Setup UI blocks
with gr.Blocks(title="TurboFan RUL Estimator", theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    # 🚀 Turbofan Engine RUL Predictor
    Upload your engine sensor readings OR autofill dummy values to get Remaining Useful Life (RUL).
    """)

    with gr.Tab("📄 Upload File"):
        file_input = gr.File(label="Upload CSV (30 rows × 21 columns)", file_types=[".csv"])
        file_predict_btn = gr.Button("🚀 Predict from File")
        file_output = gr.Number(label="Predicted RUL (cycles)", interactive=False)
        file_predict_btn.click(fn=predict_rul_from_file, inputs=file_input, outputs=file_output)

    with gr.Tab("🧪 Manual Input"):
        sensor_inputs = [
            gr.Dataframe(
                label=f"Sensor {i+1} (last 30 readings)",
                row_count=1,
                col_count=30,
                type="numpy",
                wrap=True,
            ) for i in range(21)
        ]

        with gr.Row():
            run_btn = gr.Button("🚀 Predict RUL")
            dummy_btn = gr.Button("🎲 Autofill Random Values")

        manual_output = gr.Number(label="Predicted RUL (cycles)", interactive=False)

        run_btn.click(fn=predict_rul_from_arrays, inputs=sensor_inputs, outputs=manual_output)
        dummy_btn.click(fn=generate_dummy_data, inputs=[], outputs=sensor_inputs)

app.launch(inline=False)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c1578d7be4bce368c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
